# Behavioral context recognition system in a wild.

This Kaggle note used ExtraSensory Dataset and refinign the python code in the website http://extrasensory.ucsd.edu/ and showing different machine learning algorithms used to deal with system recognitions data.




In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports
Import pandas, numpy, matplotlib,and seaborn. Then set %matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from io import StringIO;
import os;
import os.path;
import glob as glob
import warnings
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB
from skmultilearn.problem_transform import ClassifierChain
from sklearn import metrics

# EDA

    o   Defining features
    o   Defining context labels
    o   Identifying relationship between features and context labels
    o   Identifying relationship between labels
    o   Select the features to work with for target label
    o   Defining 1 target labels to work with.
    o   Cleaning missing labels and replacing NaN values with dataset samples


In [ ]:
def get_sensor_names_from_features(feature_names):
    feat_sensor_names = np.array([None for feat in feature_names]);
    for (fi,feat) in enumerate(feature_names):
        if feat.startswith('raw_acc'):
            feat_sensor_names[fi] = 'Acc';
            pass;
        elif feat.startswith('proc_gyro'):
            feat_sensor_names[fi] = 'Gyro';
            pass;
        elif feat.startswith('raw_magnet'):
            feat_sensor_names[fi] = 'Magnet';
            pass;
        elif feat.startswith('watch_acceleration'):
            feat_sensor_names[fi] = 'WAcc';
            pass;
        elif feat.startswith('watch_heading'):
            feat_sensor_names[fi] = 'Compass';
            pass;
        elif feat.startswith('location'):
            feat_sensor_names[fi] = 'Loc';
            pass;
        elif feat.startswith('location_quick_features'):
            feat_sensor_names[fi] = 'Loc';
            pass;
        elif feat.startswith('audio_naive'):
            feat_sensor_names[fi] = 'Aud';
            pass;
        elif feat.startswith('audio_properties'):
            feat_sensor_names[fi] = 'AP';
            pass;
        elif feat.startswith('discrete'):
            feat_sensor_names[fi] = 'PS';
            pass;
        elif feat.startswith('lf_measurements'):
            feat_sensor_names[fi] = 'LF';
            pass;
        else:
            raise ValueError("!!! Unsupported feature name: %s" % feat);

        pass;

    return feat_sensor_names;  

'''
Read the data (precomputed sensor-features and labels) for a user.
This function assumes the user's data file is present.
'''
def validate_column_names_are_consistent(old_column_names,new_column_names):
    if len(old_column_names) != len(new_column_names):
        raise ValueError("!!! Inconsistent number of columns.");
        
    for ci in range(len(old_column_names)):
        if old_column_names[ci] != new_column_names[ci]:
            raise ValueError("!!! Inconsistent column %d) %s != %s" % (ci,old_column_names[ci],new_column_names[ci]));
        pass;
    return;
def get_label_pretty_name(label):
    if 'FIX_walking' in label:
        return 'Walking';
    if 'FIX_running' in label:
        return 'Running';
    if 'LOC_main_workplace' in label:
        return 'At main workplace' ;
    if 'OR_indoors' in label:
        return 'Indoors';
    if 'OR_outside' in label:
        return 'Outside';
    if 'LOC_home' in label:
        return 'At home';
    if 'FIX_restaurant' in label:
        return 'At a restaurant';
    if 'OR_exercise' in label:
        return 'Exercise';
    if  'LOC_beach' in label:
        return 'At the beach';
    if 'OR_standing' in label:
        return 'Standing';
    if 'WATCHING_TV' in label:
        return 'Watching TV'
    else:
        label.replace('label:', '')
    if label.endswith('_'):
        label = label[:-1] + ')';
        pass;
    
    label = label.replace('__',' (').replace('_',' ');
    label = label[0] + label[1:].lower();
    label = label.replace('i m','I\'m');
    return label;

def get_phone_label(label):
    if label == 'FIX_walking':
        return 'Walking';
    if label == 'FIX_running':
        return 'Running';
    if label == 'LOC_main_workplace':
        return 'At main workplace';
    if label == 'OR_indoors':
        return 'Indoors';
    if label == 'OR_outside':
        return 'Outside';
    if label == 'LOC_home':
        return 'At home';
    if label == 'FIX_restaurant':
        return 'At a restaurant';
    if label == 'OR_exercise':
        return 'Exercise';
    if label == 'LOC_beach':
        return 'At the beach';
    if label == 'OR_standing':
        return 'Standing';
    if label == 'WATCHING_TV':
        return 'Watching TV'
    
    if label.endswith('_'):
        label = label[:-1] + ')';
        pass;
    
    label = label.replace('__',' (').replace('_',' ');
    label = label[0] + label[1:].lower();
    label = label.replace('i m','I\'m');
    
    # if lable is phone related then return the label
    if "Phone" not in label: 
        return label
    else:
        return False


def get_sensor_names_from_features(feature_names):
    feat_sensor_names = np.array([None for feat in feature_names]);
    for (fi,feat) in enumerate(feature_names):
        if feat.startswith('raw_acc'):
            feat_sensor_names[fi] = 'Acc';
            pass;
        elif feat.startswith('proc_gyro'):
            feat_sensor_names[fi] = 'Gyro';
            pass;
        elif feat.startswith('raw_magnet'):
            feat_sensor_names[fi] = 'Magnet';
            pass;
        elif feat.startswith('watch_acceleration'):
            feat_sensor_names[fi] = 'WAcc';
            pass;
        elif feat.startswith('watch_heading'):
            feat_sensor_names[fi] = 'Compass';
            pass;
        elif feat.startswith('location'):
            feat_sensor_names[fi] = 'Loc';
            pass;
        elif feat.startswith('location_quick_features'):
            feat_sensor_names[fi] = 'Loc';
            pass;
        elif feat.startswith('audio_naive'):
            feat_sensor_names[fi] = 'Aud';
            pass;
        elif feat.startswith('audio_properties'):
            feat_sensor_names[fi] = 'AP';
            pass;
        elif feat.startswith('discrete'):
            feat_sensor_names[fi] = 'PS';
            pass;
        elif feat.startswith('lf_measurements'):
            feat_sensor_names[fi] = 'LF';
            pass;
        else:
            raise ValueError("!!! Unsupported feature name: %s" % feat);

        pass;

    return feat_sensor_names;  

def get_features_from_data(users_df):
    for (ci,col) in enumerate(users_df.columns):
        if col.startswith('label:'):
            first_label_ind = ci;
            break;
    pass;
    feature_names = users_df.columns[1:first_label_ind];
    return np.array(feature_names)

def project_features_to_selected_sensors(feature_names,sensors_to_use):

    feature_names_arr = []
    for sensor in sensors_to_use:
        if sensor == 'Acc':
            for feature in feature_names:
                #print (type(feature))
                if (feature.startswith('raw_acc')):
                    feature_names_arr.append(feature)
        elif sensor == 'WAcc':
            for feature in feature_names:
                if (feature.startswith('watch_acceleration')):
                    feature_names_arr.append(feature)
        elif sensor == 'Gyro':
            for feature in feature_names:
                if (feature.startswith('proc_gyro')):
                    feature_names_arr.append(feature)
        elif sensor == 'Magnet':
            for feature in feature_names:
                if (feature.startswith('raw_magnet')):
                    feature_names_arr.append(feature)
        elif sensor == 'Compass':
            for feature in feature_names:
                if (feature.startswith('watch_heading')):
                    feature_names_arr.append(feature)
        elif sensor == 'Loc':
            for feature in feature_names:
                if (feature.startswith('location')):
                    feature_names_arr.append(feature)
        elif sensor == 'Aud':
            for feature in feature_names:
                if (feature.startswith('audio_naive')):
                    feature_names_arr.append(feature)
        elif sensor == 'AP':
            for feature in feature_names:
                if (feature.startswith('audio_properties')):
                    feature_names_arr.append(feature)
        elif sensor == 'PS':
            for feature in feature_names:
                if (feature.startswith('discrete')):
                    feature_names_arr.append(feature)
        elif sensor == 'LF':
            for feature in feature_names:
                if (feature.startswith('lf_measurements')):
                    feature_names_arr.append(feature)
                    
    return feature_names_arr

def estimate_standardization_params(X):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        mean_vec = np.nanmean(X,axis=0);
        std_vec = np.nanstd(X,axis=0);
        return (mean_vec,std_vec);

def standardize_features(X,mean_vec,std_vec):
    # Subtract the mean, to centralize all features around zero:
    X_centralized = X - mean_vec.reshape((1,-1));
    # Divide by the standard deviation, to get unit-variance for all features:
    # * Avoid dividing by zero, in case some feature had estimate of zero variance
    normalizers = np.where(std_vec > 0., std_vec, 1.).reshape((1,-1));
    X_standard = X_centralized / normalizers;
    return X_standard;

def get_label_names(users_df):
    # Search for the column of the first label:
    for (ci,col) in enumerate(users_df.columns):
        if col.startswith('label:'):
            first_label_ind = ci;
            break;
        pass;

    label_names = np.array(users_df.columns[first_label_ind:-1]);
    for (li,label) in enumerate(label_names):
        # In the CSV the label names appear with prefix 'label:', but we don't need it after reading the data:
        assert label.startswith('label:');
        #label_names[li] = label.replace('label:','');
        pass;
    
    return (list(label_names));

def print_accuracy_repoprt(predictions, y_test):

    accuracy = np.mean(predictions == y_test);
    # Count occorrences of true-positive, true-negative, false-positive, and false-negative:
    tp = np.sum(np.logical_and(predictions,y_test));
    tn = np.sum(np.logical_and(np.logical_not(predictions),np.logical_not(y_test)));
    fp = np.sum(np.logical_and(predictions,np.logical_not(y_test)));
    fn = np.sum(np.logical_and(np.logical_not(predictions),y_test));

    # Sensitivity (=recall=true positive rate) and Specificity (=true negative rate):
    sensitivity = float(tp) / (tp+fn);
    specificity = float(tn) / (tn+fp);

    # Balanced accuracy is a more fair replacement for the naive accuracy:
    balanced_accuracy = (sensitivity + specificity) / 2.;

    # Precision:
    # Beware from this metric, since it may be too sensitive to rare labels.
    # In the ExtraSensory Dataset, there is large skew among the positive and negative classes,
    # and for each label the pos/neg ratio is different.
    # This can cause undesirable and misleading results when averaging precision across different labels.
    precision = float(tp) / (tp+fp);
    accuracy_list = [accuracy,sensitivity,specificity,balanced_accuracy,precision]
    print("-"*10);
    print('Accuracy*:         %.2f' % accuracy);
    print('Sensitivity (TPR): %.2f' % sensitivity);
    print('Specificity (TNR): %.2f' % specificity);
    print('Balanced accuracy: %.2f' % balanced_accuracy);
    print('Precision**:       %.2f' % precision);
    print("-"*10);
    #print('* The accuracy metric is misleading - it is dominated by the negative examples (typically there are many more negatives).')
    #print('** Precision is very sensitive to rare labels. It can cause misleading results when averaging precision over different labels.')
    return accuracy_list

In [ ]:
def prepare_X_Y_for_ML(users_df):
    # prepare data for machine learning
    # 1. get all features available
    feature_names = get_features_from_data(users_df)

    # 2. get the features sensors feat from features
    feat_sensor_names = get_sensor_names_from_features(feature_names);
        
    # 3. select the sensors to use in the machine learning
    #sensors_to_use = ['Acc','WAcc'];

    # 4. get Data accoring to selected sensors with feaures;
    #feature_names_arr = []
    #feature_names_arr = project_features_to_selected_sensors(feature_names, sensors_to_use)
    X = users_df[feature_names]
    # 5. stanrdize the features substracting the mean value and dividing by standard deviation
    # so that all their values will be roughly in the same range:
    (mean_vec,std_vec) = estimate_standardization_params(X);
    X = standardize_features(X,mean_vec,std_vec);
    X[np.isnan(X)] = 0.
    # 6. X is ready for training
    # 7. Prepare Y target lables for training
    label_names = get_label_names(users_df)
    Y = users_df[label_names]
    # 8. clean nan values and converted to binary labels
    # Read the binary label values, and the 'missing label' indicators:
    trinary_labels_mat = users_df[label_names]; # This should have values of either 0., 1. or NaN
    M = np.isnan(trinary_labels_mat); # M is the missing label matrix
    Y = np.where(M,0,trinary_labels_mat) > 0.; # Y is the label matrix
    y_df = pd.DataFrame(Y)
    y_df.rename(columns=dict(enumerate(label_names, 0)), inplace = True)
    return (X,y_df,M,feature_names,label_names)

# GET the data
Try reading sample user in order to extract features and label names

In [ ]:
%%time
#sample_user = pd.read_csv('61359772-D8D8-480D-B623-7C636EAD0C81.features_labels.csv')
sample_user = pd.read_csv('/kaggle/input/extrasensory-dataset/1155FF54-63D3-4AB2-9863-8385D0BD0A13.features_labels.csv')
#sample_user = pd.read_csv('1155FF54-63D3-4AB2-9863-8385D0BD0A13.features_labels.csv')

In [ ]:
#Now let's check the features data columns before that has NAN values in certain columns "lf_measurements:temperature_ambient"
sample_user.info()

**Standardize sensor features measurements by substracting mean and deviding by SD and context lable to have only bolean variables**


In [ ]:
# process the data to get features data and context label data
(X,Y,M,feature_names,label_names) = prepare_X_Y_for_ML(sample_user)
XY = pd.concat([X, Y], axis=1, sort=False)
XY.head()

Displaying all columns in the dataset


In [ ]:
print("Number of columns in the database =", sample_user.columns.size)
for column in sample_user.columns:
    print (column)

Listing feature names available in the dataset.
features columns are all columns except the timestamp and context columns and they are 225 column

In [ ]:
print ("Number of features columns in the dataset =",feature_names.size)
print (feature_names)

In [ ]:
Y

In [ ]:

sample_user[feature_names].head()

In [ ]:
#sns.distplot(sample_user,x=sample_user.index,y='raw_acc:magnitude_stats:mean', color='red')

sensor features data after standrization process


In [ ]:
# let's check the features data after standarization, it has been set to 0 instead of NaN
#sns.displot(data=X,x=X.index,y='raw_acc:magnitude_stats:mean')
X[feature_names].head()

if we compare the histogram for the column "raw_acc:magnitude_stats:mean" it shows that standardization process worked fine and we have all values close to zero

# getting relations between context labels if any for that sample user

In [ ]:
corr = Y[label_names].corr().sort_values(by=label_names, ascending=False)
corr

we have 10 labels for this users that are missing data or have NaN values, so we will ignore them in our next step

# removing Nan from the view to focus on the interesting labels

In [ ]:
corr.dropna(how='all')

# showing a heatmap for correlation between labels will list 41 label instead of 51 for this sample user

In [ ]:
corr.dropna(how='all').style.background_gradient(cmap='coolwarm', axis=None)

* As per the heatmap shown for this sample user, there is strong coorelation between certain labels for example Lying Down, comes always with sleeping , Running comes with excercise, indoor comes with location at home, Bathing Shower comes with singing. However there are certain labels that comes only with itself and there are no chance it happens with other label, like Resturant.
* But in order to get more correlation for the whole population and not only this sample user. we read the whole users dataset all at once of 60 users with 300K+

In [ ]:
#
%%time
path = r'C:\Users\amrabouelenein\Desktop\BHCS\ExtraSensory' # use your path
csv_files = glob.glob(path + "/*.csv")

fifty_users_files = []
ten_users_files = [] 
list_user_files = []
i = 1
for file in csv_files:
    user_data = pd.read_csv(file)
    list_user_files.append(user_data)
    if i <= 50:
        fifty_users_files.append(user_data)
    else:
        ten_users_files.append(user_data)
    i = i + 1

all_users_data = pd.concat(list_user_files, axis=0, ignore_index=True)
fifty_users_data = pd.concat(fifty_users_files, axis=0, ignore_index=True)
ten_users_data = pd.concat(ten_users_files, axis=0, ignore_index=True)

In [ ]:
%%time
(X_all,Y_all,M_all,feature_names,label_names) = prepare_X_Y_for_ML(all_users_data)

In [ ]:
%%time
corr_all = Y_all[label_names].corr().sort_values(by=label_names, ascending=False)
corr_all

# from the table above we can see that the missing labels in the sample users are filled and have relation with other labels, so we can plot at heatmap to visualise the whole dataset of all 60 users

In [ ]:
#
%%time
corr_all.style.background_gradient(cmap='coolwarm', axis=None)

1. There are still labeles that come togeather with reasonble propablity like sleepting and Lying down. Byceling comes with excercise and label:STAIRS_-_GOINGUP label:STAIRS-_GOING_DOWN. the rest of labels doesn't seems to always happens togeather, and this maybe due to user didn't report multiple context label
1. let's get the most lables reported among all users to consider them in our target labels for prediction in machine learining process
1. between label we need to check the correlation between sensors features and context labels in order to use them for the maching learning excercise

In [ ]:
n_examples_per_label = np.sum(np.array(Y),axis=0)
labels_and_counts = zip(label_names,n_examples_per_label)
sorted_labels_and_counts = sorted(labels_and_counts,reverse=True,key=lambda pair:pair[1])
print ("number of examples for every context label:")
print ("-"*20)
i = 0
label_x_arr = []
label_y_arr = []
for (label,count) in sorted_labels_and_counts:
    i = i +1
    label_x_arr.append(label)
    label_y_arr.append(count)
    print (" %i : %s - %d minutes"  % (i,label,count))
    pass;

In [ ]:
#X.isnull().values.any()
labels_df = pd.DataFrame(sorted_labels_and_counts)
labels_df.rename(columns = {0:'label'}, inplace = True) 
labels_df.rename(columns = {1:'count'}, inplace = True) 
labels_df.plot(x='label', y='count', kind='bar', legend=False, grid=True, figsize=(20, 8))

* selecting the top labels with many examples recorded in our dataset.
* Choosing Walking as target context label and the 14 top lable to be included in our Multiple label classifcation stage later on
* label:OR_indoors - 184692 minutes
* label:LOC_home - 152892 minutes
* label:SITTING - 136356 minutes
* label:PHONE_ON_TABLE - 115037 minutes
* label:LYING_DOWN - 104210 minutes
* label:SLEEPING - 83055 minutes
* label:AT_SCHOOL - 42331 minutes
* label:COMPUTER_WORK - 38081 minutes
* label:OR_standing - 37782 minutes
* label:TALKING - 36293 minutes
* label:LOC_main_workplace - 33944 minutes
* label:WITH_FRIENDS - 24737 minutes
* label:PHONE_IN_POCKET - 23401 minutes
* label:FIX_walking - 22136 minutes
* label:SURFING_THE_INTERNET - 19416 minutes
* listing the availble sensors for the dataset, we extract the name from column name
* 1. Acc stands for acceloremeter
* 2. Gyro stands for Gyroscope
* 3. Magnet stands for Magnetormeter
* 4. WAcc stands for Watch acceloremeter
* 5. Compass stands for Compass
* 6. Loc stands for Location
* 7. Aud stands for Audio
* 8. AP stands for Application
* 9. PS stands for Phone State
* 10. LFstands for LF

In [ ]:
feat_sensor_names = get_sensor_names_from_features(feature_names)
print(pd.unique(feat_sensor_names))

1. Displaying a list of columns features related to certain features for acceloremeter (Acc) and Watch acceloremeter (WAcc) that can coorelate with Walking activity
1. checking the values of acceloremeters with the corresponding labels to see any coorelation between multiple lables and sensors in the same time

In [ ]:
features_of_selected_sensors =  project_features_to_selected_sensors(feature_names,['Acc', 'WAcc'])
features_of_selected_sensors

# Maching learining Part

1. logistic regression model
2. KNN model
3. Linear model

**Logistic Regression Model with label WALKING**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X[features_of_selected_sensors], Y['label:FIX_walking'], test_size=0.30, random_state=42)

In [ ]:
%time
logmodel = LogisticRegression(max_iter=200)
logmodel.fit(X_train,y_train)

In [ ]:
%time
predictions = logmodel.predict(X_test)

In [ ]:
#print(classification_report(y_test,predictions))
logmodel_results =  print_accuracy_repoprt(predictions,y_test)

**logistic regression has very good accuracy but low sensetivity for this label .69 which is not high sensetive for the label Walking.
Linear Regression
testing the model with sample user**

In [ ]:
lm = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X[features_of_selected_sensors], Y["label:FIX_walking"], test_size=0.20, shuffle=True, random_state=42)    
lm.fit(X_train,y_train)

In [ ]:
# calculate these metrics by hand!
predictions = lm.predict(X_test)

print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
#sns.displot((y_test-predictions),bins=50);

In [ ]:
#print(classification_report(y_test,predictions.round()))
lm_results =  print_accuracy_repoprt(predictions,y_test)

# linear regression model is not the right one for label Walking.
# Trying KNearest Neighbour model for label WALKING

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()


In [ ]:
scaler.fit(XY[features_of_selected_sensors])

In [ ]:
%%time
scaled_features = scaler.transform(XY[features_of_selected_sensors])

In [ ]:
%%time
df_feat = pd.DataFrame(scaled_features,columns=features_of_selected_sensors)
df_feat.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(scaled_features,XY['label:FIX_walking'],test_size=0.30, random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)

In [ ]:
%%time
knn.fit(X_train,y_train)

In [ ]:
%%time
pred = knn.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,pred))
print(classification_report(y_test,pred))
knn_results =  print_accuracy_repoprt(pred,y_test)

# Choosing a K Value
# Let's go ahead and use the elbow method to pick a good K Value!

 Create a for loop that trains various KNN models with different k values, then keep track of the error_rate for each of these models with a list. Refer to the lecture if you are confused on this step.

In [ ]:
%%time
error_rate = []

# Will take some time
for i in range(1,40):
    
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred_i = knn.predict(X_test)
    error_rate.append(np.mean(pred_i != y_test))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(range(1,40),error_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

In [ ]:
%%time
# NOW WITH K=30
knn = KNeighborsClassifier(n_neighbors=17)

knn.fit(X_train,y_train)
pred = knn.predict(X_test)

print('WITH K=17')
print('\n')
print(confusion_matrix(y_test,pred))
print('\n')
print(classification_report(y_test,pred))
knn_results =  print_accuracy_repoprt(pred,y_test)

**KNN has an excelent score for true value which is better than logistic model for the label Walking.**

In [ ]:
single_label_models_results = [logmodel_results, lm_results, knn_results]
data = single_label_models_results
x_axis = np.arange(5)


accuracy_list = ['Accuracy', 'Sensitivity (TPR)', 'Specificity (TNR)', 'Balanced_accuracy', 'Precision']

width = 0.35  # the width of the bars
fig, ax = plt.subplots(figsize=(15, 7))
rects1 = ax.bar(x_axis - width/2 , np.round(knn_results, decimals=2) , width, label='KNN')
#rects2 = ax.bar(x_axis + 00.25, np.round(lm_results, decimals=2) , width, label='Linear')
rects2 = ax.bar(x_axis + width/2, np.round(logmodel_results, decimals=2) , width, label='Logistic Model')

ax.set_xticks(x_axis)
ax.set_ylabel('Scores')
ax.set_title('Scores by by every model for single label walking')
ax.set_xticklabels(accuracy_list)
ax.legend()

def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

# KNN show much more better results over logistic regression model
# Multiple Label Classification

iterating through multiple label at one time
selection of the below labels

label:LYING_DOWN

label:LOC_home

label:LOC_main_workplace

label:SITTING

label:OR_standing

label:FIX_walking

label:SLEEPING


In [ ]:
arr_labels = ['label:FIX_walking','label:LYING_DOWN','label:LOC_home','label:LOC_main_workplace','label:SITTING','label:OR_standing', 'label:SLEEPING']
mlp_logmodel_results = list()
for label in arr_labels:
    X_train, X_test, y_train, y_test = train_test_split(X[features_of_selected_sensors], Y[label], test_size=0.30, random_state=42)    
    logmodel = LogisticRegression(max_iter=200)
    logmodel.fit(X_train,y_train)
    predictions = logmodel.predict(X_test)
    print ("********************** "+label+" ******************")
    print(classification_report(y_test,predictions))
    mlp_logmodel_results.append(print_accuracy_repoprt(predictions,y_test))

We can see that label:SLEEPING has the highest accurace among the other label with 93%

# Using pipeline for applying logistic regression and one vs rest classifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X[features_of_selected_sensors], Y, test_size=0.30, random_state=42)    

In [ ]:
%%time
# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),])
for label in arr_labels:
    print('**Processing {} context label...**'.format(label))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(X_train, y_train[label])
    
    # calculating test accuracy
    predictions = LogReg_pipeline.predict(X_test)
    print('Test accuracy is {}'.format(accuracy_score(y_test[label], predictions)))
    print("\n")

# MLP using KNN model to predict the top selected labels
# we will try with all users dataset

In [ ]:
scaler = StandardScaler()
scaler.fit(XY[features_of_selected_sensors])
scaled_features = scaler.transform(XY[features_of_selected_sensors])
df_feat = pd.DataFrame(scaled_features,columns=features_of_selected_sensors)
df_feat.head()

In [ ]:
%%time
#arr_labels = ['label:LYING_DOWN','label:LOC_home','label:LOC_main_workplace','label:SITTING','label:OR_standing','label:FIX_walking', 'label:SLEEPING']
mlp_knn_results = list()
for label in arr_labels:
    X_train, X_test, y_train, y_test = train_test_split(scaled_features,XY[label],test_size=0.30, random_state=42)
    knn = KNeighborsClassifier(n_neighbors=17)
    knn.fit(X_train,y_train)
    pred = knn.predict(X_test)
    print ("********************** "+label+" ******************")
    print('Test accuracy is {}'.format(accuracy_score(pred,y_test)))
    mlp_knn_results.append(print_accuracy_repoprt(pred,y_test))
    print("\n")

In [ ]:
#single_label_models_results = [logmodel_results, lm_results, knn_results]
#data = single_label_models_results
x_axis = np.arange(7)


#arr_labels = ['LYING_DOWN','LOC_home','LOC_main_workplace','SITTING','OR_standing','FIX_walking', 'SLEEPING']

width = 0.35  # the width of the bars
fig, ax = plt.subplots(figsize=(15, 7))

rects1 = ax.bar(x_axis - width/2 , np.round(np.array(mlp_knn_results)[:,0], decimals=3) , width, label='MLP KNN')
rects2 = ax.bar(x_axis + 00.25, np.round(np.array(mlp_logmodel_results)[:,0], decimals=3) , width, label='MLP Logistic')
#rects2 = ax.bar(x_axis + width/2, np.round(logmodel_results, decimals=2) , width, label='Logistic Model')

ax.set_xticks(x_axis)
ax.set_ylabel('Scores')
ax.set_title('Scores of accuracy  by every model for every labels')
ax.set_xticklabels(arr_labels)
ax.legend()

def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

As per the bar chart shown above, the accuracy score is better for KNN in certain labels like Walking, Standing and setting, however the logistic regression has better score for sleeping and . KNN and logitistc regression model shows pretty similar results accross the targeted labels. so let's check the other evaluators for the models TPR and TNR

In [ ]:
#single_label_models_results = [logmodel_results, lm_results, knn_results]
#data = single_label_models_results
x_axis = np.arange(7)


#arr_labels = ['LYING_DOWN','LOC_home','LOC_main_workplace','SITTING','OR_standing','FIX_walking', 'SLEEPING']

width = 0.35  # the width of the bars
fig, ax = plt.subplots(figsize=(15, 7))

rects1 = ax.bar(x_axis - width/2 , np.round(np.array(mlp_knn_results)[:,1], decimals=3) , width, label='MLP KNN')
rects2 = ax.bar(x_axis + 00.25, np.round(np.array(mlp_logmodel_results)[:,1], decimals=3) , width, label='MLP Logistic')
#rects2 = ax.bar(x_axis + width/2, np.round(logmodel_results, decimals=2) , width, label='Logistic Model')

ax.set_xticks(x_axis)
ax.set_ylabel('Scores')
ax.set_title('Scores of ** Sensetivity (TPR) **  by every model for every labels')
ax.set_xticklabels(arr_labels)
ax.legend()

def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

In [ ]:
#single_label_models_results = [logmodel_results, lm_results, knn_results]
#data = single_label_models_results
x_axis = np.arange(7)


#arr_labels = ['LYING_DOWN','LOC_home','LOC_main_workplace','SITTING','OR_standing','FIX_walking', 'SLEEPING']

width = 0.35  # the width of the bars
fig, ax = plt.subplots(figsize=(15, 7))

rects1 = ax.bar(x_axis - width/2 , np.round(np.array(mlp_knn_results)[:,2], decimals=3) , width, label='MLP KNN')
rects2 = ax.bar(x_axis + 00.25, np.round(np.array(mlp_logmodel_results)[:,2], decimals=3) , width, label='MLP Logistic')
#rects2 = ax.bar(x_axis + width/2, np.round(logmodel_results, decimals=2) , width, label='Logistic Model')

ax.set_xticks(x_axis)
ax.set_ylabel('Scores')
ax.set_title('Scores of ** Specificity (TNR) **  by every model for every labels')
ax.set_xticklabels(arr_labels)
ax.legend()

def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

In [ ]:
np.array(mlp_knn_results)[:,0]